# Sklearn
Download the model
The model should be saved as `model.joblib` and the metadata should be `metadata.yaml`

In [10]:
# Note: comment out one of the options
# option 1: use your own model
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import numpy as np
clf = LogisticRegression(solver="liblinear", multi_class="ovr")
X, y = datasets.load_iris(return_X_y=True)
clf.fit(X, y)
clf.predict(X[0,:].reshape(1,-1))
from joblib import dump, load
dump(clf, 'model.joblib')

# option 2: use models from Seldon core
# %%bash
# mc config host add gcs https://storage.googleapis.com "" "" 

# mc mb minio-seldon/iris -p
# mc cp gcs/seldon-models/v1.14.0-dev/sklearn/iris/model.joblib minio/minio-seldon/iris/
# mc cp gcs/seldon-models/v1.14.0-dev/sklearn/iris/metadata.yaml minio/minio-seldon/iris/

['model.joblib']

In [11]:
!mc ls minio/minio-seldon/iris

[2022-05-10 13:16:52 UTC]   906B STANDARD model.joblib


In [3]:
# import job
!mc mb minio/minio-seldon/iris -p
!mc cp ./models/model.joblib minio/minio-seldon/iris/
!mc ls minio/minio-seldon/iris

Bucket created successfully `minio/minio-seldon/iris`.
...el.joblib:  906 B / 906 B ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 110.85 KiB/s 0s[2022-05-10 13:16:52 UTC]   906B STANDARD model.joblib


# Rclone parameters
We pass the rclone parameters for getting data from minio using secret

In [12]:
%%writefile rclone-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting rclone-secret.yaml


In [13]:
!kubectl apply -f ./rclone-secret.yaml

secret/seldon-rclone-secret configured


In [1]:
%%writefile sklearn.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn-nosidecar-1
spec:
  name: iris
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://minio-seldon/iris/
      envSecretRefName: seldon-rclone-secret
      name: classifier
    name: default
    replicas: 1

Overwriting sklearn.yaml


In [2]:
!kubectl apply -f sklearn.yaml

seldondeployment.machinelearning.seldon.io/sklearn-nosidecar-1 created


In [18]:
!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:32000/seldon/default/sklearn-prof-daqiq/api/v1.0/predictions \
   -H "Content-Type: application/json"

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[9.912315378486697e-07,0.0007015931307746079,0.9992974156376876]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.13.1"}}}


## Istio
to add notebook with istio sidecar (for visualization) add the following label

In [5]:
%%writefile sklearn.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn-prof-daqiq
spec:
  name: iris
  predictors:
  - graph:
      implementation: SKLEARN_SERVER
      modelUri: s3://minio-seldon/iris/
      envSecretRefName: seldon-rclone-secret
      name: classifier
    labels:
      sidecar.istio.io/inject: "true"
    name: default
    replicas: 1

Overwriting sklearn.yaml
